# 面向中文短文本实体链接比赛方案

## 比赛介绍
### 赛题背景
近年来，随着深度学习的重燃以及海量大数据的支撑，知识图谱领域迎来了蓬勃发展，百度拥有全球最大的中文知识图谱，拥有数亿实体、千亿事实，具备丰富的知识标注与关联能力，不仅构建了通用知识图谱，还构建了汉语语言知识图谱、关注点图谱、以及包含业务逻辑在内的行业知识图谱等多维度图谱。我们希望通过开放百度的数据，邀请学界和业界的青年才俊共同推进算法进步，激发更多灵感和火花。

面向中文短文本的实体链指，简称 EL（Entity Linking），是NLP、知识图谱领域的基础任务之一，即对于给定的一个中文短文本（如搜索 Query、微博、对话内容、文章/视频/图片的标题等），EL将其中的实体与给定知识库中对应的实体进行关联。

传统的实体链指任务主要是针对长文档，长文档拥有在写的上下文信息能辅助实体的歧义消解并完成链指。相比之下，针对中文短文本的实体链指存在很大的挑战，主要原因如下：

（1）口语化严重，导致实体歧义消解困难；

（2）短文本上下文语境不丰富，须对上下文语境进行精准理解；

（3）相比英文，中文由于语言自身的特点，在短文本的链指问题上更有挑战。

本次评测任务旨在借助实体链指技术，拓展其对应的AI智能应用需求，并将技术成果实践于更多的现实场景。

### 赛题说明
输入：
中文短文本以及该短文本中的实体集合。

输出：
输出文本指中文短文本的实体链指结果。每个结果包含：实体mention、在中文短文本中的位置偏移、其在给定知识库中的id，如果为NIL情况，需要再给出实体的上位概念类型

输入示例：
```json
{
    "text_id":"1",
    "text":"《琅琊榜》海宴_【原创小说|权谋小说】",
    "mention_data":[
        {
            "mention":"琅琊榜",
            "offset":"1"
        },
        {
            "mention":"海宴",
            "offset":"5"
        },
        {
            "mention":"原创小说",
            "offset":"9"
        },
        {
            "mention":"权谋小说",
            "offset":"14"
        }
    ]
}
```

输出示例：
```json
{
    "text_id":"1",
    "text":"《琅琊榜》海宴_【原创小说|权谋小说】",
    "mention_data":[
        {
            "kb_id":"2135131",
            "mention":"琅琊榜",
            "offset":"1"
        },
        {
            "kb_id":"10572965",
            "mention":"海宴",
            "offset":"5"
        },
        {
            "kb_id":"215143",
            "mention":"原创小说",
            "offset":"9"
        },
        {
            "kb_id":" NIL_Work ",
            "mention":"权谋小说",
            "offset":"14"
        }
    ]
}
```

[比赛详细说明地址](https://aistudio.baidu.com/aistudio/competition/detail/58)

### 重点问题

1. 指称项分类问题。如果指称项在知识库中没有找到对应的实体，如何确定对应的上位概念关系呢？
2. 候选实体召回。如何从知识库的大量实体中，筛选出和输入文本有关的实体集合？
3. 实体排序。如何根据候选实体的结果和已知的输入文本信息和实体信息，对候选实体进行排序，选出最契合短文本指称项的知识库实体。

针对以上问题下面提供解决思路。

## 比赛思路

主要思路参照：
[实体链指比赛方案分享](https://aistudio.baidu.com/aistudio/projectdetail/1331020?channelType=0&channel=0)
的思路建立两阶段的分类模型，简化实现和加入自己思考的小细节。


主要流程描述：

1. 指称项分类模型。输入指称项和包含指称项的原文本拼接，输出指称项的类型。
2. 候选实体召回。遍历知识库，把知识库实体的关系转化为：“实体id-实体信息” 和 “实体指称-实体id” 的映射。从原文本的mention文本出发，根据“实体指称-实体id”匹配实体文本召回候选实体。
3. 候选实体特征提取。首先用指称项分类模型，来预测输入数据的指称项的实体类型。根据候选实体召回结果，对于有召回的实体：用“实体id-实体信息”提取处实体信息，按顺序组织实体信息的文本内容后拼接原始文本丰富实体的语义信息，最后把指称项的实体类型加入构成完整的实体候选集合。对于无召回的实体，就无需进行候选实体排序，直接与排序结果进行后处理整合即可。
4. 候选实体排序模型。输入标记指称项的原始文本和候选实体信息的拼接，输出指称项和候选实体的匹配程度。
5. 后处理。整合“候选实体排序模型”和“指称项分类模型”的输出结果，生成最后的提交结果。

主要流程如下图:

![](https://ai-studio-static-online.cdn.bcebos.com/26e9d50bc1d649b0b7c99191fca5446673243a84c4d44c7b8c274c0399d540cb)


### 数据处理方式
#### 指称项分类二阶段训练实现
对于一个mention对应多个type的情况，只取对应的第一个type。

首先把数据分为非NIL和NIL实体，然后在此基础上分别把数据分成5折输入模型进行训练。

第一阶段：选取非NIL的数据进行训练2个epoch，学习率5e-6

第二阶段：选取NIL的数据进行训练5个epoch，学习率1e-5

#### 召回负采样
知识库中同名实体统计
| stat| value |
| -------- | -------- |
| count     | 105159.000000|
|mean|3.085024|
|std|7.330572|
|min|1.000000|
|max|283.000000|



知识库中同名实体数量差异

![](https://ai-studio-static-online.cdn.bcebos.com/a9cc7ab6ad3147afbda51ef052058b3a587adbfc9ed444d6952fa68ed8a6132e)

x是同名实体按数量排序的id，y是同名实体的个数，不同实体召回的数量级不同，会导致样本不均衡。

从知识库来讲，同名实体好多都是人，平均同名实体数量是 3，所以候选实体排序进行负采样一般控制在 3 左右比较好。

[更加详细的数据分析请点击这里](https://aistudio.baidu.com/aistudio/projectdetail/1351768)

### 指称项分类
主要采用基于BERT 的文本对分类思路，输入text_a：指称项文本，text_b：原始短文本，经过BERT编码后，取出CLS token 的特征向量进行softmax分类，输出指称项的类别。

模型结构如下图：
![](https://ai-studio-static-online.cdn.bcebos.com/f142544e280c44509ed801638cc57aa5f3908b0b0aa2441fbd2efcd8f036e1fd)

1. 指称项分类模型采用两阶段训练的方式，两阶段的架构相同，仅输入数据和超参数不同。第一阶段集中训练非NIL部分的数据，第二部分在此基础上用NIL部分的数据以较小的学习率进行微调。
2. 最后采用5折交叉验证ERNIE 和 roberta-wwm-large-ext 平均融合。

[指称项分类训练和推理代码](https://aistudio.baidu.com/studio/project/partial/verify/1458477/75678f8955044967a7fd9e3d086f73d9)

### 候选实体召回 和 特征提取
这里的候选实体召回实际上就是以知识库中实体词库为基础做关键词匹配。我们根据文本中的mention在知识库中找到 “subject 或者 “alias” 与其匹配的实体id作为候选的实体集合。因为有召回的指称项其实有可能是NIL type，我们还需要构造一个NIL实体。训练数据的NIL实体就是对应实体id的类型，预测数据的NIL实体就是根据 ①指称项分类 预测结果得到。

特征提取主要是根据候选实体的结果，根据实体id一般实体按照 1.类型 2.摘要 3.义项描述 4.标签 其它...等等 按顺序，拼接它的上下文信息，以便于和输入文本的mention进行匹配。

### 候选实体排序
这里的模型架构实际上和 ①指称项分类 类似，只是输入和输出有点略微不同，输入是文本对text_a：标记mention位置后的文本，text_b：提取的候选实体的描述信息。经过BERT编码后，取出CLS token 的特征向量进行softmax分类，输出候选实体和mention的匹配程度。

模型整体结构如下图：
![](https://ai-studio-static-online.cdn.bcebos.com/ffacaa9448764ca9bafb67f368082578a8bc2a7997e74b1c9b2fba477b7514cd)

这里采用ERNIE对构造的文本对进行二分类。根据得到的匹配分数排序选出top1实体作为最后匹配的实体。

[候选实体排序训练和推理代码](https://aistudio.baidu.com/studio/project/partial/verify/1418258/121066d9ad03487b9d577748f7aefc55)

### 模型调整策略
初赛
仅采用ERNIE进行指称项分类和候选实体排序

复赛
在初赛的基础上尝试，指称项分类加入5-fold cross-validtion，排序模型不变。
在上面的基础上模型融合，指称项分类进行了ERNIE和 roberta-wwm-large-ext 的融合，排序模型不变。

### 实验分析


| 模型 |  F1 | Precision | Recall|
| -------- | -------- | -------- |-------- |
| ERNIE only 初赛     | 0.86739     | 0.86739     |0.86739     |
|CV+ERNIE(mention clf)->ERNIE(rank) 复赛|0.90326	|0.90333|0.90319|
|CV+ERNIE+roberta(metion clf)->ERNIE(rank) 复赛|0.90383	|0.9039|0.90376|


### 总结
工程
1. 没有全局考虑方案再动手，代码混乱，复用性差。
2. 框架不熟悉，未实现对抗训练，未实现动态的负采样。

方案流程
首先建立科学的本地验证集，编写符合指标的评测脚本。评估召回、排序的结果根据表现改进。

指称项分类

1. 根据统计选择过滤类别。
2. 考虑模型在 token 级别的特征抽取改进。

实体链接

1. 两阶段模型 召回-排序 可以改进成 召回-粗排-精排
2. 实体链接的目标是在相似的、迷惑性高的召回结果里面，选择最准确的实体。负采样的概率指导应该是以统计指称项对应实体的概率，标注数据中指称项对应实体的概率越高，被抽到的概率应该越大。同样地，在预测时遇到召回太多的情况下，也可以根据这个逻辑过滤实体。
整体流程框架应该考虑模型融合，不同召回结果多路召回如何融合cv，精排的输入固定这样就比较好进行特征融合和cv。

#### 飞桨使用体验
- paddlehub 预训练模型结合下游任务 finetune 方便，更改预训练模型只需要改支持的预训练模型名字即可。
- 官方仓库例子涵盖了多种通用的nlp任务，可以参照后官方例子自己实现多种通用任务训练。
- 可能是我对框架不熟悉，我感觉局限是非通用任务或者需要修改层的实现不太方便。但是我体验过paddlepaddle2.0高阶api，和市面上其它框架一样支持子类化组网等特性。这种情况会改善。
- 初学者建议在aistudio搜索自己感兴趣的项目，结合官方demo做项目实验，之后涉猎比较复杂的底层代码实现自己需要的功能。

#### 飞桨参赛体验
- 有往届选手分享和资料，入门引导项目有两个官方baseline，一个大牛的比赛方案分享，帮助选手开阔思路
- 比赛问题定义清晰，官方已经划分好了验证集，写好离线评测脚本，对新手友好。
- aistudio平台提供免费算力，参赛可得纪念品和奖金，还能结识一些一起参加比赛的朋友。
- 对于初学者aistudio平台提供了丰富的课程和项目代码，从参加飞桨比赛开始进行实践会令你进步很多。
- 对于ai开发者，参加飞桨比赛可以帮助你迅速掌握飞桨这个框架，同时也可以把自己平时积累的知识应用到比赛解决方案上。

## 参考资料
[1. 实体链指比赛方案分享](https://aistudio.baidu.com/aistudio/projectdetail/1331020?channelType=0&channel=0)

[2. paddlehub 文本分类demo](https://github.com/PaddlePaddle/PaddleHub/tree/release/v1.7/demo/text_classification)

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 